In [11]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('./images'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



./images/maksssksksss828.png
./images/maksssksksss669.png
./images/maksssksksss449.png
./images/maksssksksss72.png
./images/maksssksksss96.png
./images/maksssksksss648.png
./images/maksssksksss182.png
./images/maksssksksss586.png
./images/maksssksksss447.png
./images/maksssksksss364.png
./images/maksssksksss306.png
./images/maksssksksss216.png
./images/maksssksksss409.png
./images/maksssksksss699.png
./images/maksssksksss393.png
./images/maksssksksss17.png
./images/maksssksksss210.png
./images/maksssksksss29.png
./images/maksssksksss703.png
./images/maksssksksss242.png
./images/maksssksksss469.png
./images/maksssksksss237.png
./images/maksssksksss143.png
./images/maksssksksss64.png
./images/maksssksksss629.png
./images/maksssksksss95.png
./images/maksssksksss299.png
./images/maksssksksss248.png
./images/maksssksksss166.png
./images/maksssksksss650.png
./images/maksssksksss800.png
./images/maksssksksss608.png
./images/maksssksksss770.png
./images/maksssksksss520.png
./images/makssskskss

In [12]:

import os
import xml.etree.ElementTree as et
import re
import pandas as pd

dic = {"image": [],"Dimensions": []}
for i in range(1,116):
	dic[f'Object {i}']=[]
print("Generating data in CSV format....")

for file in os.listdir("./annotations"):
    row = []
    xml = et.parse("./annotations/"+file) 
    root = xml.getroot()
    img = root[1].text
    row.append(img)
    h,w = root[2][0].text,root[2][1].text
    row.append([h,w])

    for i in range(4,len(root)):
        temp = []
        temp.append(root[i][0].text)
        for point in root[i][5]:
            temp.append(point.text)
        row.append(temp)
    for i in range(len(row),119):
        row.append(0)
    for i,each in enumerate(dic):
        dic[each].append(row[i])
df = pd.DataFrame(dic)

Generating data in CSV format....


In [13]:
df.head()

,image,Dimensions,Object 1,Object 2,Object 3,Object 4,Object 5,Object 6,Object 7,Object 8,Object 9,Object 10,Object 11,Object 12,Object 13,Object 14,Object 15,Object 16,Object 17,Object 18,Object 19,Object 20,Object 21,Object 22,Object 23,Object 24,Object 25,Object 26,Object 27,Object 28,Object 29,Object 30,Object 31,Object 32,Object 33,Object 34,Object 35,Object 36,Object 37,Object 38,...,Object 76,Object 77,Object 78,Object 79,Object 80,Object 81,Object 82,Object 83,Object 84,Object 85,Object 86,Object 87,Object 88,Object 89,Object 90,Object 91,Object 92,Object 93,Object 94,Object 95,Object 96,Object 97,Object 98,Object 99,Object 100,Object 101,Object 102,Object 103,Object 104,Object 105,Object 106,Object 107,Object 108,Object 109,Object 110,Object 111,Object 112,Object 113,Object 114,Object 115
0,maksssksksss133.png,"[301, 400]","[with_mask, 100, 179, 221, 318]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,maksssksksss612.png,"[400, 295]","[with_mask, 173, 88, 296, 231]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,maksssksksss741.png,"[267, 400]","[mask_weared_incorrect, 111, 40, 151, 77]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,maksssksksss586.png,"[400, 267]","[with_mask, 111, 106, 146, 164]","[with_mask, 194, 115, 224, 164]","[with_mask, 22, 120, 50, 161]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,maksssksksss66.png,"[380, 400]","[with_mask, 179, 81, 267, 173]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
# view summary of dataset
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853 entries, 0 to 852
Columns: 117 entries, image to Object 115
dtypes: object(117)
memory usage: 779.8+ KB


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pandas as pd
import glob
import os
import cv2
import random as rand

image_directories = sorted(glob.glob(os.path.join("./images","*.png")))
j=0
classes = ["without_mask","mask_weared_incorrect","with_mask"]
labels = []
data = []

print("Extracting each data into respective label folders....")
for idx,image in enumerate(image_directories):
    img  = cv2.imread(image)
    #scale to dimension
    X,Y = df["Dimensions"][idx]
    cv2.resize(img,(int(X),int(Y)))
    #find the face in each object
    for obj in df.columns[3:]:
        info = df[obj][idx]
        if info!=0:
            label = info[0]
            info[0] = info[0].replace(str(label), str(classes.index(label)))
            info=[int(each) for each in info]
            face = img[info[2]:info[4],info[1]:info[3]]
            if((info[3]-info[1])>40 and (info[4]-info[2])>40):
                try:
                    face = cv2.resize(face, (224, 224))
                    face = img_to_array(face)
                    face = preprocess_input(face)
                    data.append(face)
                    labels.append(label)
                    if(label=="mask_weared_incorrect"):
                        data.append(face)
                        labels.append(label)

                except:
                    pass


                
print("Done!")

Extracting each data into respective label folders....
Done!


In [16]:
data = np.array(data, dtype="float32")
labels = np.array(labels)
labels

array(['with_mask', 'with_mask', 'mask_weared_incorrect',
       'mask_weared_incorrect', 'with_mask', 'with_mask',
       'mask_weared_incorrect', 'mask_weared_incorrect', 'with_mask',
       'mask_weared_incorrect', 'mask_weared_incorrect', 'with_mask',
       'mask_weared_incorrect', 'mask_weared_incorrect',
       'mask_weared_incorrect', 'mask_weared_incorrect', 'with_mask',
       'with_mask', 'with_mask', 'without_mask', 'with_mask', 'with_mask',
       'with_mask', 'with_mask', 'mask_weared_incorrect',
       'mask_weared_incorrect', 'with_mask', 'without_mask', 'with_mask',
       'without_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask',
       'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask',
       'with_mask', 'with_mask', 'with_mask', 'with_mask',
       'mask_weared_incorrect', 'mask_weared_incorrect', 'with_mask',
       'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask',
       'with_mask', 'with_mask', 'with_mask', 'without_mask',


In [17]:
lb = LabelEncoder()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [18]:
aug = ImageDataGenerator(
    zoom_range=0.1,
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
    )

In [19]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

9412608/9406464 [==============================] - 0s 0us/step


In [20]:
INIT_LR = 1e-4
EPOCHS = 50
BS = 1

In [21]:
len(data)

340

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853 entries, 0 to 852
Columns: 117 entries, image to Object 115
dtypes: object(117)
memory usage: 779.8+ KB


In [23]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.3, stratify=labels, random_state=42)

In [24]:
print(trainX)

[[[[ 0.6627451   0.5686275   0.5764706 ]
   [ 0.654902    0.56078434  0.5686275 ]
   [ 0.6156863   0.52156866  0.5372549 ]
   ...
   [ 0.45882356  0.43529415  0.43529415]
   [ 0.45098042  0.43529415  0.43529415]
   [ 0.45098042  0.43529415  0.43529415]]

  [[ 0.6627451   0.5686275   0.5764706 ]
   [ 0.654902    0.56078434  0.5686275 ]
   [ 0.6156863   0.52156866  0.5294118 ]
   ...
   [ 0.45882356  0.4431373   0.4431373 ]
   [ 0.45098042  0.43529415  0.43529415]
   [ 0.45098042  0.43529415  0.43529415]]

  [[ 0.654902    0.5686275   0.5686275 ]
   [ 0.654902    0.56078434  0.56078434]
   [ 0.6156863   0.52156866  0.5294118 ]
   ...
   [ 0.4666667   0.45098042  0.45098042]
   [ 0.45882356  0.4431373   0.4431373 ]
   [ 0.45882356  0.4431373   0.4431373 ]]

  ...

  [[ 0.4431373   0.09803927 -0.05098039]
   [ 0.43529415  0.09019613 -0.05882353]
   [ 0.39607847  0.05098045 -0.09803921]
   ...
   [ 0.9372549   0.94509804  0.96862745]
   [ 0.9372549   0.94509804  0.96862745]
   [ 0.9372549  

In [ ]:
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

[INFO] compiling model...
[INFO] training head...
Epoch 1/50
238/238 [==============================] - 21s 87ms/step - loss: 0.8127 - accuracy: 0.7437 - val_loss: 0.5796 - val_accuracy: 0.8235
Epoch 2/50
238/238 [==============================] - 20s 86ms/step - loss: 0.6373 - accuracy: 0.8025 - val_loss: 0.5703 - val_accuracy: 0.8333
Epoch 3/50
238/238 [==============================] - 19s 82ms/step - loss: 0.5395 - accuracy: 0.8109 - val_loss: 0.5769 - val_accuracy: 0.8333
Epoch 4/50
 51/238 [=====>........................] - ETA: 11s - loss: 0.6648 - accuracy: 0.7843

In [ ]:
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=32)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

# serialize the model to disk
print("[INFO] saving mask detector model...")

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()

ValueError: ignored